<a href="https://colab.research.google.com/github/cyberust/HumanSynergyAnalysis_JP/blob/main/HumanSynergyAnalysis_JP_TEST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
# ======================================================================
#
#  人のシナジー分析システム V4.2 - 最終感度改善版
#
#  V4からの改善点:
#  - スキルスコアの正規化処理を導入し、感度を向上。
#    各人のスキルプロファイルの「形状」の違いを明確化する。
#
# ======================================================================

# ===============================================================
# STEP 1: ライブラリのインストール
# ===============================================================
!pip install pandas numpy scipy networkx plotly scikit-learn textblob vaderSentiment --quiet
print("✅ ライブラリのインストールが完了しました。")

# ===============================================================
# STEP 2: ライブラリのインポート
# ===============================================================
import os, re, shutil, time, math
import pandas as pd
import numpy as np
import networkx as nx
import plotly.graph_objects as go
import plotly.express as px
from datetime import datetime
from scipy.integrate import odeint
from sklearn.metrics.pairwise import cosine_similarity
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from itertools import combinations
import warnings
warnings.filterwarnings('ignore')
from google.colab import drive
print("✅ ライブラリのインポートが完了しました。")

# ===============================================================
# STEP 3: 分析パラメータの一元管理
# ===============================================================
CONFIG = {
    'data': {
        'base_path': "/content/drive/MyDrive/",
        'names': ["arale_cohen", "yanay_geva", "yasuyuki_sakane"],
        'display_names': ["Arale Cohen", "Yanay Geva", "Yasuyuki Sakane"]
    },
    'profile_analysis': {
        'experience_default': 10,
        'network_keywords': [
            'network', 'ecosystem', 'connections', 'board member', 'partner',
            'collaboration', 'global', 'silicon valley'
        ],
        'network_multiplier': 15,
        'skill_keywords': {
            'AI/ML': [
                'artificial intelligence', 'machine learning', 'deep learning', 'ai', 'ml', 'nlp'
            ],
            'Product': [
                'product', 'development', 'launch', 'strategy', 'planning', 'roadmap',
                'product management', 'product owner', 'go-to-market', 'gtm',
                'product marketing', 'lifecycle', 'solution', 'service', 'design',
                'architecture', 'implementation', 'offering', 'functionality', 'feature',
                'Managed Security Services', 'MSS', 'Security Operations Center', 'SOC',
                'Cyber Fusion Center', 'CFC', 'DevSecOps', 'SBOM', 'vulnerability management',
                'threat intelligence', 'CTI', 'adaptive defense', 'threat modeling',
                'threat hunting', 'Operational Technology', 'OT', 'SIEM',
                'endpoint behavior auditing', 'grid-based web infrastructure',
                'Microsoft Cluster Services', 'MSCS', 'service prototyping',
                'operational automation', 'product-market fit', 'ILP',
                'firmware lifecycle management', 'public key infrastructure',
                'low-code automation', 'digital intelligence', 'forensics',
                'proactive', 'resilience', 'transformation', 'infrastructure', 'security',
                'data extraction', 'data analysis', 'mobile forensics'
            ],
            'Finance': [
                'finance', 'fintech', 'venture capital', 'investment', 'funding', 'vc', 'roi',
                'M&A', 'profitability', 'cash flow'
            ],
            'Leadership': [
                'leadership', 'management', 'team', 'cto', 'ceo', 'executive', 'director',
                'CISO', 'CAIO', 'CoEs', 'Centers of Excellence', 'structural innovation',
                'organizational intelligence', 'mentor', 'advisor', 'governance',
                'thought leader', 'project manager', 'PMO', 'organization', 'organizational change'
            ],
            'International': [
                'global', 'international', 'cross-cultural', 'multilingual', 'fluent',
                'cross-border', 'localization', 'APAC', 'bilateral', 'global team', 'overseas HQ',
                'structural translator', 'bilingual'
            ]
        }
    },
    'synergy_calculation': {
        'weights': {'complementarity': 0.5, 'experience': 0.25, 'network': 0.25},
        'skill_normalization_total': 300,
        'cultural_backgrounds': {'Arale Cohen': 'Western', 'Yanay Geva': 'Western', 'Yasuyuki Sakane': 'Eastern'},
        'diversity_bonus': 1.2,
    },
    'dynamic_model': {
        'alpha': np.array([0.4, 0.35, 0.45]), 'beta_multiplier': 0.1, 'K': np.array([12.0, 12.0, 12.0]),
        'initial_state': [1.0, 1.0, 1.0], 'time_horizon': 24
    },
    'game_theory': {'synergy_value_multiplier': 5},
    'prediction': {
        'monte_carlo_simulations': 1000, 'investment_cost': 1000000,
        'revenue_scale_factor': 10000, 'npv_discount_rate': 0.05
    },
    'output': {'base_path': "/content/drive/MyDrive/", 'folder_prefix': "synergy_full_analysis_"}
}
print("✅ 分析パラメータの定義が完了しました。")

# ===============================================================
# STEP 4: 各種分析クラスの定義
# ===============================================================

class DataLoader:
    def __init__(self, config): self.config = config['data']
    def load_profiles(self):
        mount_point = '/content/drive'
        if os.path.exists(mount_point) and not os.path.ismount(mount_point): shutil.rmtree(mount_point)
        drive.mount(mount_point, force_remount=True)
        profiles = {}
        for name, display_name in zip(self.config['names'], self.config['display_names']):
            try:
                with open(f"{self.config['base_path']}{name}_profile.txt", 'r', encoding='utf-8') as f:
                    profiles[display_name] = f.read()
            except FileNotFoundError: profiles[display_name] = ""
        return profiles

class KeywordProfileAnalyzer:
    def __init__(self, config):
        self.config = config['profile_analysis']
        self.sentiment_analyzer = SentimentIntensityAnalyzer()
    def analyze(self, profiles_dict):
        return {name: {'skills_raw_count': self._extract_skills_raw_count(text),
                       'experience_years': self._extract_experience(text),
                       'network_strength': self._extract_network_strength(text),
                       'sentiment': self.sentiment_analyzer.polarity_scores(text)['compound']}
                for name, text in profiles_dict.items()}
    def _extract_skills_raw_count(self, text):
        text_lower = text.lower()
        return {cat: sum(text_lower.count(kw) for kw in kws) for cat, kws in self.config['skill_keywords'].items()}
    def _extract_experience(self, text):
        matches = re.findall(r'(\d+)\+?\s*years?', text.lower())
        return max([int(m) for m in matches]) if matches else self.config['experience_default']
    def _extract_network_strength(self, text):
        score = sum(text.lower().count(k) for k in self.config['network_keywords'])
        return min(score * self.config['network_multiplier'], 100)

class HeuristicSynergyCalculator:
    def __init__(self, config): self.config = config['synergy_calculation']
    def calculate(self, analysis_data):
        names = list(analysis_data.keys())

        # V4.2の核心：スキルスコアの正規化
        raw_counts = np.array([list(analysis_data[n]['skills_raw_count'].values()) for n in names])
        sum_counts = raw_counts.sum(axis=1, keepdims=True)
        sum_counts[sum_counts == 0] = 1 # ゼロ除算を回避
        normalized_skills = (raw_counts / sum_counts) * self.config['skill_normalization_total']

        # 正規化後のスキルベクトルで補完性を計算
        comp_matrix = 1 - cosine_similarity(normalized_skills)

        experiences = [analysis_data[n]['experience_years'] for n in names]
        exp_matrix = np.array([[np.exp(-0.1 * abs(exp_i - exp_j)) if i != j else 0 for j, exp_j in enumerate(experiences)] for i, exp_i in enumerate(experiences)])
        networks = [analysis_data[n]['network_strength'] for n in names]
        net_matrix = np.array([[(net_i * net_j) / 10000.0 if i != j else 0 for j, net_j in enumerate(networks)] for i, net_i in enumerate(networks)])

        w = self.config['weights']
        total_synergy = (w['complementarity'] * comp_matrix + w['experience'] * exp_matrix + w['network'] * net_matrix)

        backgrounds = self.config['cultural_backgrounds']
        for i, n1 in enumerate(names):
            for j, n2 in enumerate(names):
                if backgrounds.get(n1) != backgrounds.get(n2): total_synergy[i, j] *= self.config['diversity_bonus']

        np.fill_diagonal(total_synergy, 0)
        return {'total_synergy': total_synergy, 'skill_scores_normalized': {names[i]: normalized_skills[i] for i in range(len(names))}}

class DynamicSystemModel:
    def __init__(self, config): self.config = config['dynamic_model']
    def simulate(self, synergy_matrix):
        c=self.config; t=np.linspace(0,c['time_horizon'],c['time_horizon']*4); s=odeint(self._system_dynamics,c['initial_state'],t,args=(c['alpha'],synergy_matrix*c['beta_multiplier'],c['K'])); return t,s
    def _system_dynamics(self, state, t, alpha, beta, K):
        x=np.array(state); return(alpha*x+np.dot(beta,x))*(1-x/K)

class PredictionModel:
    def __init__(self, config): self.config = config['prediction']
    def run_all(self, growth_solution, time_axis, synergy_matrix):
        forecast = self._forecast_business_metrics(growth_solution, time_axis, synergy_matrix)
        monte_carlo = self._monte_carlo_simulation(growth_solution)
        roi = self._calculate_roi_metrics(forecast)
        return {'forecast': forecast, 'monte_carlo': monte_carlo, 'roi': roi}
    def _forecast_business_metrics(self, growth_data, time_axis, synergy_matrix):
        return pd.DataFrame([{'time_months': time_axis[t], 'revenue_growth_rate': (np.sum(growth_data[t]) + np.sum(synergy_matrix * np.outer(growth_data[t], growth_data[t]))/2) * 20} for t in range(len(time_axis))])
    def _monte_carlo_simulation(self, growth_data):
        return pd.DataFrame([{'final_performance': np.sum(growth_data[-1] * np.random.normal(1, 0.1, growth_data.shape[1])),
                              'success': 1 if np.sum(growth_data[-1]) > np.sum(growth_data[0]) * 1.5 else 0}
                             for _ in range(self.config['monte_carlo_simulations'])])
    def _calculate_roi_metrics(self, forecast_df):
        rev=forecast_df['revenue_growth_rate']*self.config['revenue_scale_factor']; cum_rev=rev.cumsum(); cost=self.config['investment_cost']
        roi=(cum_rev-cost)/cost*100; payback=next((t for t,r in zip(forecast_df['time_months'],cum_rev)if r>=cost),None)
        dr=self.config['npv_discount_rate']/12; npv=np.sum(rev/(1+dr)**np.arange(len(rev)))-cost; return{'roi_over_time':roi,'payback_period_months':payback,'npv':npv}

class GameTheoryAnalyzer:
    def __init__(self, config): self.config = config['game_theory']
    def calculate_shapley(self, analysis_data, synergy_matrix):
        names,n=list(analysis_data.keys()),len(analysis_data); sv=np.zeros(n)
        for i in range(n):
            for cs in range(n):
                for c in combinations(range(n),cs):
                    if i in c:continue
                    mc=self._get_coalition_value(list(c)+[i],analysis_data,synergy_matrix)-self._get_coalition_value(list(c),analysis_data,synergy_matrix)
                    sv[i]+=mc*(math.factorial(cs)*math.factorial(n-cs-1))/math.factorial(n)
        return sv
    def _get_coalition_value(self,indices,analysis_data,synergy_matrix):
        if not indices:return 0
        names=list(analysis_data.keys());bv=sum(analysis_data[names[i]]['experience_years']+analysis_data[names[i]]['network_strength']/10.0 for i in indices)
        sy_v=sum(synergy_matrix[i,j]for i in indices for j in indices if i!=j);return bv+sy_v*self.config['synergy_value_multiplier']

class VisualizationGenerator:
    def __init__(self, config): self.config=config
    def generate_all_figures(self, results):
        figures,names={},self.config['data']['display_names']; skill_cats=list(self.config['profile_analysis']['skill_keywords'].keys())
        fig_r=go.Figure(); [fig_r.add_trace(go.Scatterpolar(r=list(results['synergy']['skill_scores_normalized'][n]),theta=skill_cats,fill='toself',name=n))for n in names]; fig_r.update_layout(title='V4.2: Skill Profile (Normalized)'); figures['skill_radar']=fig_r
        figures['synergy_heatmap']=px.imshow(results['synergy']['total_synergy'],x=names,y=names,text_auto='.3f',title='V4.2: Overall Synergy Matrix')
        df_g=pd.DataFrame(results['simulation_solution'],columns=names); df_g['Time (Months)']=results['simulation_time_axis']; figures['growth_trajectory']=px.line(df_g,x='Time (Months)',y=names,title='V4.2: Dynamic Growth Simulation')
        figures['monte_carlo']=px.histogram(results['predictions']['monte_carlo'],x='final_performance',nbins=50,title='V4.2: Monte Carlo Simulation')
        df_roi=pd.DataFrame({'ROI (%)':results['predictions']['roi']['roi_over_time'],'Time (Months)':results['predictions']['forecast']['time_months']}); figures['roi_analysis']=px.line(df_roi,x='Time (Months)',y='ROI (%)',title='V4.2: ROI Over Time')
        figures['shapley_values']=px.bar(x=names,y=results['shapley_values'],text_auto='.2f',title='V4.2: Value Contribution (Shapley)'); return figures

# ===============================================================
# STEP 5: 実行パイプラインの定義と実行
# ===============================================================
class AnalysisPipeline:
    def __init__(self, config):
        self.config=config; self.data_loader=DataLoader(config); self.profile_analyzer=KeywordProfileAnalyzer(config); self.synergy_calculator=HeuristicSynergyCalculator(config)
        self.dynamic_model=DynamicSystemModel(config); self.prediction_model=PredictionModel(config); self.game_theory_analyzer=GameTheoryAnalyzer(config)
        self.viz_generator=VisualizationGenerator(config); self.run_timestamp=datetime.now().strftime("%Y%m%d_%H%M%S")
    def run(self):
        print("--- パイプライン実行開始 V4.2 (最終感度改善版) ---")
        output_folder=os.path.join(self.config['output']['base_path'],f"{self.config['output']['folder_prefix']}{self.run_timestamp}"); os.makedirs(output_folder,exist_ok=True)
        print(f"✅ 出力先フォルダ準備完了: '{output_folder}' (同期のため5秒待機)"); time.sleep(5)
        profiles=self.data_loader.load_profiles(); analysis_data=self.profile_analyzer.analyze(profiles); synergy_results=self.synergy_calculator.calculate(analysis_data)
        time_axis,solution=self.dynamic_model.simulate(synergy_results['total_synergy']); predictions=self.prediction_model.run_all(solution,time_axis,synergy_results['total_synergy'])
        shapley_values=self.game_theory_analyzer.calculate_shapley(analysis_data,synergy_results['total_synergy'])
        results={'config':self.config,'analysis':analysis_data,'synergy':synergy_results,'simulation_time_axis':time_axis,'simulation_solution':solution,'predictions':predictions,'shapley_values':shapley_values}
        results['figures']=self.viz_generator.generate_all_figures(results); self._save_artifacts(results,output_folder); print("\n--- ✅ パイプライン実行完了 ---"); return results
    def _save_artifacts(self,results,output_folder):
        html=self._create_combined_html(results['figures']); open(os.path.join(output_folder,"dashboard.html"),'w',encoding='utf-8').write(html); print("✅ 統合ダッシュボードを保存しました。")
        report=self._generate_report(results); open(os.path.join(output_folder,"report.md"),'w',encoding='utf-8').write(report); print("✅ 詳細レポートを保存しました。")
    def _create_combined_html(self,figures):
        html="<html><head><title>Synergy Dashboard V4.2</title><script src='https://cdn.plot.ly/plotly-latest.min.js'></script></head><body>"
        for n,f in figures.items():html+=f"<h2>{f.layout.title.text}</h2><div>{f.to_html(full_html=False,include_plotlyjs=False)}</div>"
        return html+"</body></html>"
    def _generate_report(self,results):
        names=self.config['data']['display_names']; roi=results['predictions']['roi']['roi_over_time'].iloc[-1]; pb=results['predictions']['roi']['payback_period_months']
        sp=results['predictions']['monte_carlo']['success'].mean()*100; r=f"# Synergy Report V4.2\n\n## Executive Summary\n- Success Probability: {sp:.1f}%\n- Final ROI: {roi:.1f}%\n- Payback Period: {pb if pb else 'N/A'} months\n\n"
        r+="## Shapley Values\n"; r+="".join([f"- {n}: {v:.2f}\n" for n,v in zip(names,results['shapley_values'])]); return r

# パイプラインを実行
pipeline = AnalysisPipeline(CONFIG)
results = pipeline.run()

✅ ライブラリのインストールが完了しました。
✅ ライブラリのインポートが完了しました。
✅ 分析パラメータの定義が完了しました。
--- パイプライン実行開始 V4.2 (最終感度改善版) ---
✅ 出力先フォルダ準備完了: '/content/drive/MyDrive/synergy_full_analysis_20250620_225159' (同期のため5秒待機)
Mounted at /content/drive
✅ 統合ダッシュボードを保存しました。
✅ 詳細レポートを保存しました。

--- ✅ パイプライン実行完了 ---
